In [173]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [174]:
train_values = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')

In [175]:
set_train = train_values.merge(train_labels)
set_train.drop( columns=["building_id"], inplace=True)
#set_train.set_index("building_id")
set_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [176]:
columnas_seleccionadas = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'land_surface_condition',
                         'has_superstructure_cement_mortar_stone']
train_values_subset = set_train[columnas_seleccionadas]
train_labels = set_train.iloc[:,-1]

In [177]:
train_values_subset = pd.get_dummies(train_values_subset)

In [178]:
rf_model = RandomForestClassifier(random_state=1)

In [179]:
rf_model.fit(train_values_subset, train_labels.values.ravel())

RandomForestClassifier(random_state=1)

In [180]:
preds = rf_model.predict(train_values_subset)
f1_score(train_labels, preds, average='micro')

0.7068890756367013

In [181]:
test_values = pd.read_csv('test_values.csv',index_col='building_id')

In [182]:
test_values_subset = test_values[columnas_seleccionadas]
test_values_subset = pd.get_dummies(test_values_subset)

In [183]:
predictions = rf_model.predict(test_values_subset)

In [184]:
submission_format = pd.read_csv('submission_format.csv',index_col='building_id')

In [185]:
my_submission = pd.DataFrame(data=predictions, columns=submission_format.columns,
                            index=submission_format.index)

In [186]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [187]:
my_submission.to_csv('submision.csv')

In [188]:
!head submision.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,2
896100,3
343471,2
